In [16]:
import pandas as pd
import glob
import re
datapath = "./Input/"
types = ["BodyWeightSquats", "PushUps"]
extension = "*.csv"



In [45]:
main_df = pd.DataFrame()

for count, cur in enumerate(types):
    current_glob = glob.glob(datapath+cur+"/"+extension)
    df = pd.DataFrame()
    li = []

    for filename in current_glob:
        df = pd.read_csv(filename, index_col=None, header=0)
        p = re.compile("\\\\v_(.*)\.csv")
        filename = p.findall(filename)[0]
        df.insert(0, 'example', filename)
        df.insert(0, 'exercise', cur)
        li.append(df)

    frame = pd.concat(li, axis=0, ignore_index=True)
    if count == 0:
        main_df = pd.DataFrame(columns=frame.columns, data=frame)
    else:
        main_df = pd.concat([main_df, frame])

print(main_df.head(1000))


             exercise                   example  frame    NOSE.x    NOSE.y  \
0    BodyWeightSquats  BodyWeightSquats_g01_c01      0  0.468669  0.069361   
1    BodyWeightSquats  BodyWeightSquats_g01_c01      1  0.468662  0.069879   
2    BodyWeightSquats  BodyWeightSquats_g01_c01      2  0.468611  0.070022   
3    BodyWeightSquats  BodyWeightSquats_g01_c01      3  0.468610  0.069943   
4    BodyWeightSquats  BodyWeightSquats_g01_c01      4  0.468484  0.069916   
..                ...                       ...    ...       ...       ...   
995  BodyWeightSquats  BodyWeightSquats_g02_c01     52  0.485500  0.114159   
996  BodyWeightSquats  BodyWeightSquats_g02_c01     53  0.487887  0.114337   
997  BodyWeightSquats  BodyWeightSquats_g02_c01     54  0.492444  0.114524   
998  BodyWeightSquats  BodyWeightSquats_g02_c01     55  0.498318  0.116184   
999  BodyWeightSquats  BodyWeightSquats_g02_c01     56  0.502815  0.126060   

       NOSE.z    NOSE.v  LEFT_EYE_INNER.x  LEFT_EYE_INNER.y  LE